<a href="https://colab.research.google.com/github/skhabiri/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module1-rnn-and-lstm/LS_DS17_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *Data Science Unit 4 Sprint 3 Module 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
import requests
import pandas as pd

In [2]:
url = "https://github.com/skhabiri/ML-DeepLearning/raw/main/data/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data[:500]

'The Project Gutenberg eBook of The Complete Works of William Shakespeare, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this eBook or online at\r\nwww.gutenberg.org. If you are not located in the United States, you\r\nwill have to check the laws of the country'

In [3]:
toc = [l.strip() for l in data.split('\r\n')[44:130:2]]
{id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

{0: {'title': 'THE TRAGEDY OF ANTONY AND CLEOPATRA', 'start': -99},
 1: {'title': 'AS YOU LIKE IT', 'start': -99},
 2: {'title': 'THE COMEDY OF ERRORS', 'start': -99},
 3: {'title': 'THE TRAGEDY OF CORIOLANUS', 'start': -99},
 4: {'title': 'CYMBELINE', 'start': -99},
 5: {'title': 'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK', 'start': -99},
 6: {'title': 'THE FIRST PART OF KING HENRY THE FOURTH', 'start': -99},
 7: {'title': 'THE SECOND PART OF KING HENRY THE FOURTH', 'start': -99},
 8: {'title': 'THE LIFE OF KING HENRY THE FIFTH', 'start': -99},
 9: {'title': 'THE FIRST PART OF HENRY THE SIXTH', 'start': -99},
 10: {'title': 'THE SECOND PART OF KING HENRY THE SIXTH', 'start': -99},
 11: {'title': 'THE THIRD PART OF KING HENRY THE SIXTH', 'start': -99},
 12: {'title': 'KING HENRY THE EIGHTH', 'start': -99},
 13: {'title': 'KING JOHN', 'start': -99},
 14: {'title': 'THE TRAGEDY OF JULIUS CAESAR', 'start': -99},
 15: {'title': 'THE TRAGEDY OF KING LEAR', 'start': -99},
 16: {'title': 'LOVE

In [4]:
"""
\r (Carriage Return) → moves the cursor to the beginning of the line without advancing to the next line
\n (Line Feed) → moves the cursor down to the next line without returning to the beginning of the line — In a *nix environment \n moves to the beginning of the line.
\r\n (End Of Line) → a combination of \r and \n
"""

url = "https://github.com/skhabiri/ML-DeepLearning/raw/main/data/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text

data = data.split('\r\n')

# Title of the writings
toc = [l.strip() for l in data[44:130:2]]

# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [5]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2,THE COMEDY OF ERRORS,17172,20372,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4,CYMBELINE,30347,30364,CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...


In [6]:
df_toc.shape

(43, 4)

In [7]:
len(df_toc['text'][1])

136678

In [8]:
df_toc['text']

0                                                      
1     AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2     THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3     THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4     CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...
5     THE TRAGEDY OF HAMLET, PRINCE OF DENMARK\r\n\r...
6     THE FIRST PART OF KING HENRY THE FOURTH\r\n\r\...
7     THE SECOND PART OF KING HENRY THE FOURTH\r\n\r...
8                                                      
9     THE LIFE OF KING HENRY V\r\n\r\n\r\n\r\nConten...
10    THE SECOND PART OF KING HENRY THE SIXTH\r\n\r\...
11    THE THIRD PART OF KING HENRY THE SIXTH\r\n\r\n...
12                     KING HENRY THE EIGHTH\r\n\r\n...
13      KING JOHN. O cousin, thou art come to set mi...
14    THE TRAGEDY OF JULIUS CAESAR\r\n\r\n\r\n\r\nCo...
15    THE TRAGEDY OF KING LEAR\r\n\r\n\r\n\r\nConten...
16    LOVE’S LABOUR’S LOST\r\n\r\nDramatis Personae....
17                                              

In [9]:
test = []
for txt in df_toc['text']:
  for x in txt:
    test.append(x)
test[:20]

['A',
 'S',
 ' ',
 'Y',
 'O',
 'U',
 ' ',
 'L',
 'I',
 'K',
 'E',
 ' ',
 'I',
 'T',
 '\r',
 '\n',
 '\r',
 '\n',
 '\r',
 '\n']

In [10]:
# data = df_toc['text'][1:].values
# data.shape

In [11]:
df_toc['text'][1][:500]

'AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r\n\r\n  DUKE, living in exile\r\n  FREDERICK, his brother, and usurper of his dominions\r\n  AMIENS, lord attending on the banished Duke\r\n  JAQUES,   "      "       "  "     "      "\r\n  LE BEAU, a courtier attending upon Frederick\r\n  CHARLES, wrestler to Frederick\r\n  OLIVER, son of Sir Rowland de Boys\r\n  JAQUES,   "   "  "    "     "  "\r\n  ORLANDO,  "   "  "    "     "  "\r\n  ADAM,   servant to Oliver\r\n  DENNIS,     "     "   "\r\n  TOUCHSTONE, the court jester\r\n  SI'

In [12]:
# removing \r\n and " from the pd.series and store as a list of strings
data = []
# txt is each of 43 chapters
for txt in df_toc['text']:
  # x is each character in a chapter
  # white spaces are not removed
  t = [x.replace('\r', ' ').replace('\n', ' ').replace('"', '') for x in txt]
  data.append("".join(t))
print(len(data))
# data is a list of 43 items. Each chapter's \r and \n is removed, but still has multiple consecutive white spaces
data[1][:500]

43


'AS YOU LIKE IT      DRAMATIS PERSONAE.      DUKE, living in exile    FREDERICK, his brother, and usurper of his dominions    AMIENS, lord attending on the banished Duke    JAQUES,                                 LE BEAU, a courtier attending upon Frederick    CHARLES, wrestler to Frederick    OLIVER, son of Sir Rowland de Boys    JAQUES,                       ORLANDO,                      ADAM,   servant to Oliver    DENNIS,                 TOUCHSTONE, the court jester    SIR OLIVER MARTEXT, a v'

In [13]:
# For each chapter converts multiple spaces into one space
data = [' '.join(data[i].split()) for i in range(len(data))]

In [14]:
data[1][:500]

'AS YOU LIKE IT DRAMATIS PERSONAE. DUKE, living in exile FREDERICK, his brother, and usurper of his dominions AMIENS, lord attending on the banished Duke JAQUES, LE BEAU, a courtier attending upon Frederick CHARLES, wrestler to Frederick OLIVER, son of Sir Rowland de Boys JAQUES, ORLANDO, ADAM, servant to Oliver DENNIS, TOUCHSTONE, the court jester SIR OLIVER MARTEXT, a vicar CORIN, shepherd SILVIUS, WILLIAM, a country fellow, in love with Audrey A person representing HYMEN ROSALIND, daughter to '

In [15]:
data[0]

''

In [16]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os
import re

In [17]:
print(len(data))
# connect all the 43 chapters with a space character into a long flattened text
# then make the text lower case as well
text = ' '.join(data).lower()

43


In [18]:
# remove all non-related characters
text = re.sub("[^a-z A-Z.:;?'!,`-]", "", text)

In [19]:
text[:500]

' as you like it dramatis personae. duke, living in exile frederick, his brother, and usurper of his dominions amiens, lord attending on the banished duke jaques, le beau, a courtier attending upon frederick charles, wrestler to frederick oliver, son of sir rowland de boys jaques, orlando, adam, servant to oliver dennis, touchstone, the court jester sir oliver martext, a vicar corin, shepherd silvius, william, a country fellow, in love with audrey a person representing hymen rosalind, daughter to'

In [20]:
# get a list of all used characters to be used as bag of items, but remove the duplicate characters
chars = list(set(text))

char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

dict_size = len(chars)
print(dict_size)
" ".join(chars)

36


"? n u x k o b w v -   , ! z ` h ; l t : f ' i q a j y e c d g p m . r s"

In [21]:
char_int

{'?': 0,
 'n': 1,
 'u': 2,
 'x': 3,
 'k': 4,
 'o': 5,
 'b': 6,
 'w': 7,
 'v': 8,
 '-': 9,
 ' ': 10,
 ',': 11,
 '!': 12,
 'z': 13,
 '`': 14,
 'h': 15,
 ';': 16,
 'l': 17,
 't': 18,
 ':': 19,
 'f': 20,
 "'": 21,
 'i': 22,
 'q': 23,
 'a': 24,
 'j': 25,
 'y': 26,
 'e': 27,
 'c': 28,
 'd': 29,
 'g': 30,
 'p': 31,
 'm': 32,
 '.': 33,
 'r': 34,
 's': 35}

Encode the text to integer numbers and create sequences of data

### Creating character sequences as training data

In [22]:
# sequence length
timestep = 40
# stepping
step = 20

# iterate through each character of the text
encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One character for each element of sequence

for i in range(0, len(encoded) - timestep, step):
    sequences.append(encoded[i : i + timestep])
    # next_char refers to the character encoding right after the last character of the sequence in encoded list
    next_char.append(encoded[i + timestep])
    
print('sequences: ', len(sequences), len(encoded)/step)

sequences:  696793 696794.7


### Functions and callbacks

In [27]:
def sample(preds):
    """
    It normalizes the array of preds to proba (with their sum equal to 1)
    Then it picks the index of a character based on a random draw considering the proba array weight
    """
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    
    # Null operation
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    
    # Normalize to the sum of one for the array of probabilities
    preds = exp_preds / np.sum(exp_preds)
    
    probas = np.random.multinomial(1, preds, 1)
    
    # Returns the indices of the maximum values along an axis.
    return np.argmax(probas)

In [28]:
def shape_seq(sentence, timestep, features, embedding=False):
    """
    create a sequence that has an input dimension compatible with NN input layer.
    The supported input layers are:
    LSTM: output x.shape=[1, timestep, features]
    Embedding: output x.shape = [1, timestep]
    inputs:
    sentence: string sentence of any length to feed into model.predict()
    timestep: pads the sentence to a sequence of exactly timestep
    features: size of dictionary of words, aka input features
    embedding: whether the input layer is embedding or LSTM
    """
    
    sentence = sentence.lower()
    # make see a fixed length padded with space if smaller
    sequence = sentence.rjust(timestep)[:timestep]  
    
    if embedding:
        # encode the characters into integers
        x = [char_int[c] for c in sequence]
        x = np.array(x)
        # model treats the input as 1 input sequence of dynamic length
        # x.shape=(1, timestep)
        x = np.expand_dims(x,0)

    else:
        x = np.zeros((1, timestep, features))
        for t, char in enumerate(sequence):
            x[0, t, char_int[char]] = 1
    
    return x

In [29]:
# Predict and convert text back into characters
def generate_text(model, seed, nextsize, timestep=timestep, features=dict_size, embedding=False):
    """
    model: trained model
    seed: some phrase for model to generate the next characters
    timestep: timestep size of one input sequence to be
    nextsize: number of the next characters to generate
    embedding: if the input layer is embedding or LSTM
    """
    
    # pad left of the sequence with blank space or slice its right side
    sequence = seed.lower().rjust(timestep)[:timestep] 
    
    generated = ''
    generated += sequence
    model.reset_states()

    for _ in range(nextsize):
        x = shape_seq(sequence, timestep, features, embedding)
        
        # pred is a numpy array  
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds)
        
        next_char = int_char[next_index]
        
        # update the sequence by moving one character forward
        sequence = sequence[1:] + next_char
        
        generated += next_char

    return generated

In [30]:
def print_text(epoch_f, logs_f):
    """ Function invoked at end of each epoch. Prints generated text.
    LambdaCallback passes epoch integer number and logs dictionary of metrics to this function.
    """
    epoch_f
    print()
    print('----- Generating text after Epoch: %d' % epoch_f)
    
    # Random prompt to grab a 40 character sample seed
    start_index = random.randint(0, len(text) - timestep - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + timestep]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        # 400 is the length of generated text
        
        x_pred = shape_seq(sentence, timestep, dict_size, embedding=(model!=model_lstm))
            
        # Predict the next step (character)
        # preds is an array of length dict_size 
        # with all the elements zero except only one of them 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        # update the seed by moving one character forward
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

### Neural Network Architecture

In [24]:
timestep, dict_size

(40, 36)

### Input layer - LSTM:
With LSTM as the input layer we need to specify:
* input_shape=[timesteps, fetures] or
* input_shape = [None, features] or
* input_dim = features
> In prediction of all three cases, the sequence could have bigger or smaller step size and it would still work with some warning.

Create X, y for LSTM layer,  (batch_size, timestep=40, features=dict_size)

In [23]:
x = np.zeros((len(sequences), timestep, dict_size), dtype=np.bool)
y = np.zeros((len(sequences), dict_size), dtype=np.bool)
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
    # y[sequence, next character after sequence in embeded]   
    y[i, next_char[i]] = 1

In [25]:
model_lstm = Sequential()

# input method 1
model_lstm.add(LSTM(128, input_shape=(timestep, dict_size)))

# # input method 2
# model_lstm.add(LSTM(128, input_shape=(None, dict_size)))

# # input method 3
# model_lstm.add(LSTM(128, input_dim=dict_size))



# output is the next character after the sequence
model_lstm.add(Dense(dict_size, activation='softmax'))

model_lstm.compile(loss='categorical_crossentropy', optimizer='adam')

In [26]:
model_lstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 36)                4644      
Total params: 89,124
Trainable params: 89,124
Non-trainable params: 0
_________________________________________________________________


In [31]:
# on_epoch_end event epoch and logs are returned to the custom function inside Lambdacallback
model = model_lstm
lcb = LambdaCallback(on_epoch_end=print_text)


# fit the model
model_lstm.fit(x, y,
          batch_size=8192,
          epochs=10,
          callbacks=[lcb])

Epoch 1/10
86/86 [==============================] - 195s 2s/step - loss: 3.1834

----- Generating text after Epoch: 0
----- Generating with seed: " had stomach for them all. desdemona. al"
 had stomach for them all. desdemona. alehlfustohve ltt eiunsn,i knihie nur,  a pbmeicwod r ;nw e enooesptdstn ifue o in r,rnloe au ehkheoa,  v ogewstmon,ti e ,h nhees  s wiihasthleylehlv gd. faitut o nofoae ods  rtgn.irehoowuaci oeio kesmskolpatrsan aewoygsta: dkein t t l  htbd do w yuie  een foi; a ui rbit  saftirdeoym;e tlw t  fntainas  eott   aoah e bi i.tn uyreho wtigitothlrfoygelohehlgt lcsllb taensn   e?jhei taenaitvb y fmi ami o
Epoch 2/10
86/86 [==============================] - 192s 2s/step - loss: 2.8162

----- Generating text after Epoch: 1
----- Generating with seed: "emn'd desires access to you. angelo. hat"
emn'd desires access to you. angelo. hatheo ssbtourtcougto: henp n, hre oirsgsp toaok gest yressnm fed,y tsmsa eo ire mreind geen s nan oee hae,, tettifu iyo tauur oh tousi hd aur, 

In [32]:
model_lstm.save('ShakespeareBot_lstm')

INFO:tensorflow:Assets written to: ShakespeareBot_lstm/assets


INFO:tensorflow:Assets written to: ShakespeareBot_lstm/assets


In [33]:
from tensorflow import keras
model_loaded = keras.models.load_model('ShakespeareBot_lstm')

In [34]:
seed = "In the darkest night of the year, "
len(seed)

34

* Note that the trained model works with sequences which have smaller or bigger time step compared to the trained timestep as well

In [35]:
generate_text(model_loaded, seed, nextsize=500, timestep=timestep+5, features=dict_size, embedding=False)

"           in the darkest night of the year, ralisinsh. i to frezire! teeleattarior, on athis moswis wemiole pore in usalian. thou you culeming hoy foo shead sish. endmy sabe to kert weer of the four, dercackd. thars: is uneming ixtongo merles, woons not the gotito frold of ty a aint. supround bat yio moust of illo saon to the enediel cunster cffilce eraver it dike. gice to pfou i speak farke of aloun'd owwy to thear hxteen havrs beigicsunca as. inor metal tors thrighy hicesdary. hice; enser woll fo thak aqoie, higla. so gentes you arke se"

### Input layer - Embedding:

With Embedding as input layer, we can specify different ways:
1. input_dim=features or
2. input_dim=features, input_length=stepsize or
3. input_dim=features, input_shape=(stepsize,)
> In prediction of all three cases, the sequence could have bigger or smaller step size and it would still work with some warning.

In [36]:
# for embedding layer we need to use sequence to fix the size of input
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence

# Pad sequences so all are equal
seq = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=timestep)
type(seq), seq.shape

(numpy.ndarray, (696793, 40))

In [37]:
# Build the model: with embedding
from keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import Bidirectional, Embedding

model_embed = Sequential()

# define input layer
# # method1, timestep will be derived dynamically
# model_embed.add(Embedding(output_dim=64, input_dim=dict_size))

# # method2, timestep of the sequence is fixed:
# # The query input still can be in different timesteps and we would only get warning
model_embed.add(Embedding(input_dim=dict_size, output_dim = 64, input_length=timestep))

# # method3, use input_shape to define timestep
# model_embed.add(Embedding(output_dim=64, input_dim=dict_size, input_shape=(timestep,)))



model_embed.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model_embed.add(Dense(dict_size, activation='softmax'))

model_embed.compile(loss='categorical_crossentropy', optimizer='adam')

In [38]:
model_embed.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 64)            2304      
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 36)                4644      
Total params: 72,996
Trainable params: 72,996
Non-trainable params: 0
_________________________________________________________________


In [39]:
model = model_embed
lcb = LambdaCallback(on_epoch_end=print_text)

# fit the model
model_embed.fit(seq, y, batch_size=8192, epochs=2, verbose=10, callbacks=[lcb])

Epoch 1/2

----- Generating text after Epoch: 0
----- Generating with seed: " ambitious constance would not cease til"
 ambitious constance would not cease tilhhnnii ra ht  t.iesecl oomden yrw.iaue, gmlmfweia,cae, sifeu ea  o cbh hr,ce,uiep adtotnmlao;ensnu oe sled  eikiw varo i.ea,tweauwo faesm  lwqntutuc;   howu he lco  hh'hstrf me  wnnaieatn  teosda mreido ue  r n   aoecaemeiehokho.io ao n m isos evmightyrihiiaooq fsaiossaaontitywhhuo .ps asednh issa lcnresntrwoe.  ra fateeboapysre noohvm oh.eltuttfbh b rner'ettwn osbwcpl iowaot ali o.shbidnmee vas s
Epoch 2/2

----- Generating text after Epoch: 1
----- Generating with seed: " to another man, and do not shear the fl"
 to another man, and do not shear the fl mhe dit hfhe m tior od upatetaliuoad rdr ye yee y cosie mlgtoth moneeciin fiuolacis otwept v moieeiaff y maf stlhewwa toai n rnsh. ole. pitegon t usqorerai chine on rolakoe ov muui.n ocecestolo woejye is f thaws,itiae anl reraut t nsd mitrhds teat agen te tlus t, yohuacr uk moc c

In [40]:
model_embed.save('ShakespeareBot_embed3')

INFO:tensorflow:Assets written to: ShakespeareBot_embed3/assets


INFO:tensorflow:Assets written to: ShakespeareBot_embed3/assets


In [41]:
from tensorflow import keras
model_loaded = keras.models.load_model('ShakespeareBot_embed3')

In [42]:
generate_text(model_loaded, seed, nextsize=100, timestep=timestep+5, features=dict_size, embedding=True)

'           in the darkest night of the year, s wos ane mocabiknat acet aguld me tonhi y as n naneuc lgashek won d uoagd todlt ibl dgik fsr theb l'

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN